## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
attrition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EnvironmentSatisfaction   1470 non-null   int64 
 8   HourlyRate                1470 non-null   int64 
 9   JobInvolvement            1470 non-null   int64 
 10  JobLevel                  1470 non-null   int64 
 11  JobRole                   1470 non-null   object
 12  JobSatisfaction           1470 non-null   int64 
 13  MaritalStatus             1470 non-null   object
 14  NumCompaniesWorked      

In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]



In [5]:
# Create a list of at least 10 column names to use as X data

features = ['Age', 'Education', 'JobSatisfaction', 'DistanceFromHome', 
            'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'NumCompaniesWorked', 'YearsAtCompany','YearsSinceLastPromotion']


# Create X_df using your selected columns
X_df = attrition_df[features]


# Show the data types for X_df
X_df.dtypes



Age                         int64
Education                   int64
JobSatisfaction             int64
DistanceFromHome            int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
NumCompaniesWorked          int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
dtype: object

In [6]:
# Convert your X data to numeric data types however you see fit
X_df.loc[:, 'OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

# X_df['OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)


In [7]:
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, random_state=78)


In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)


In [9]:
# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = encoder.transform(y_train[['Department']])
department_test_encoded = encoder.transform(y_test[['Department']])

In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Transform the 'Attrition' column in training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
num_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = tf.keras.layers.Input(shape=(num_features,))

# Create at least two shared layers
shared_layer1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
shared_layer2 = tf.keras.layers.Dense(64, activation='relu')(shared_layer1)


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer
num_departments = department_train_encoded.shape[1]

# Create the hidden layer
department_hidden = tf.keras.layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = tf.keras.layers.Dense(num_departments, activation='softmax', name='department_output')(department_hidden)


In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer
num_attrition_categories = attrition_train_encoded.shape[1]
# Create the hidden layer
attrition_hidden = tf.keras.layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = tf.keras.layers.Dense(num_attrition_categories, activation='softmax', name='attrition_output')(attrition_hidden)


In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, department_output])

# Compiling the model
model.compile(
    optimizer='adam',
    loss={
        'attrition_output': 'categorical_crossentropy',  # Use categorical_crossentropy because attrition is one-hot encoded
        'department_output': 'categorical_crossentropy'  # Use categorical_crossentropy for one-hot encoded department labels
    },
    metrics={
        'attrition_output': ['accuracy'],
        'department_output': ['accuracy']
    }
)

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          1408        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 64)           8256        ['dense[0][0]']                  
                                                                                                  
 dense_3 (Dense)                (None, 32)           2080        ['dense_1[0][0]']                
                                                                                              

In [15]:
history = model.fit(
    x=X_train_scaled,  # Scaled feature data
    y={
        'attrition_output': attrition_train_encoded,  # One-hot encoded targets for attrition
        'department_output': department_train_encoded  # One-hot encoded targets for department
    },
    validation_data=(X_test_scaled, {
        'attrition_output': attrition_test_encoded,  # One-hot encoded validation targets for attrition
        'department_output': department_test_encoded  # One-hot encoded validation targets for department
    }),
    epochs=50,  # Number of training epochs
    batch_size=32,  # Size of each batch of data
    verbose=1  # Verbosity mode: 1 for progress bar
)



Epoch 1/50
35/35 [==============================] - 1s 9ms/step - loss: 1.4458 - attrition_output_loss: 0.5407 - department_output_loss: 0.9052 - attrition_output_accuracy: 0.7568 - department_output_accuracy: 0.5635 - val_loss: 1.2043 - val_attrition_output_loss: 0.3602 - val_department_output_loss: 0.8441 - val_attrition_output_accuracy: 0.8723 - val_department_output_accuracy: 0.6168
Epoch 2/50
35/35 [==============================] - 0s 2ms/step - loss: 1.1756 - attrition_output_loss: 0.4117 - department_output_loss: 0.7638 - attrition_output_accuracy: 0.8358 - department_output_accuracy: 0.6661 - val_loss: 1.1709 - val_attrition_output_loss: 0.3284 - val_department_output_loss: 0.8426 - val_attrition_output_accuracy: 0.8804 - val_department_output_accuracy: 0.6168
Epoch 3/50
35/35 [==============================] - 0s 2ms/step - loss: 1.1379 - attrition_output_loss: 0.3846 - department_output_loss: 0.7533 - attrition_output_accuracy: 0.8485 - department_output_accuracy: 0.6661 - v

In [16]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    x=X_test_scaled,
    y={
        'attrition_output': attrition_test_encoded,
        'department_output': department_test_encoded
    },
    verbose=1  # Show progress
)

print(f"Test results - Loss: {test_results[0]} - Attrition Accuracy: {test_results[1]} - Department Accuracy: {test_results[2]}")



12/12 [==============================] - 0s 1ms/step - loss: 2.0909 - attrition_output_loss: 0.6810 - department_output_loss: 1.4099 - attrition_output_accuracy: 0.8478 - department_output_accuracy: 0.5245
Test results - Loss: 2.0908772945404053 - Attrition Accuracy: 0.6809520125389099 - Department Accuracy: 1.4099253416061401


In [17]:
# Print the accuracy for both department and attrition
# Print the accuracies for both outputs
print(f"Attrition Accuracy: {test_results[3]*100:.2f}%")
print(f"Department Accuracy: {test_results[4]*100:.2f}%")


Attrition Accuracy: 84.78%
Department Accuracy: 52.45%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not always be the best metric for evaluating models on this data, particularly if there's class imbalance, as it could lead to misleading high scores by favoring the majority class. Metrics like precision, recall, F1-score, or ROC-AUC are more informative for assessing model performance in such scenarios, as they consider the balance between different types of classification errors and class distributions.
2. I used the softmax activation function for the output layers of both the 'Attrition' and 'Department' predictions because it is ideal for multi-class classification problems, providing a probability distribution across the possible categories. This function helps in predicting the likelihood of each category, making it suitable for both binary (one-hot encoded) and multi-class outputs in the neural network.
3. To improve the model, consider using techniques like cross-validation for more robust training, incorporating dropout layers to reduce overfitting, and experimenting with different architectures or hyperparameters. Additionally, gathering more data or employing advanced feature engineering techniques, such as interaction terms or embedding layers for categorical variables, might enhance predictive performance.